#[Your_name]

<h1 style="text-align: center;">Text Classification<h1>

Prepare libraries

In [135]:
# here put every import you need e.g. import nltk
# it's better to load what you need from the package by from [] import [] instead of import the whole package
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import RegexpTokenizer
import matplotlib.pyplot as plt
import seaborn as sns
from keras.models import Sequential
from keras import layers
from keras.layers import Dense, Conv1D, AveragePooling1D, Flatten, Dropout
from keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer
import re
import numpy as np
from cleantext import clean
from arabicstopwords.stopwords_lexicon import stopwords_lexicon
import arabicstopwords.arabicstopwords as stop_words
import nltk
from nltk.stem.isri import ISRIStemmer
from wordcloud import WordCloud
from arabic_reshaper import arabic_reshaper
from bidi.algorithm import get_display
from nltk import bigrams, trigrams
from collections import Counter
from snowballstemmer import stemmer
from tashaphyne.stemming import ArabicLightStemmer
from farasa.segmenter import FarasaSegmenter
from farasa.stemmer import FarasaStemmer
from matplotlib.font_manager import FontProperties
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, balanced_accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf

from nltk.tokenize import word_tokenize


download 

Download data

In [136]:
# !wget 'https://drive.google.com/uc?export=download&id=1cMSjxa3nA706LIZDEhwMpaVRMY2IX9P0' -O 'data.zip'

In [137]:
# !pip install Arabic-Stopwords
# 2.2 nltk.download('punkt') # open vpn
# 2.3 !pip install arabic-reshaper
# 2.4 nltk.download('stopwords')
# 3.8 !pip install snowballstemmer
# 3.8 !pip install Tashaphyne
# 3.8 !pip install -U farasapy

In [138]:
# !unzip data.zip

<h2 dir="rtl">مثال عن كيفية تنظيف حلول الطلبات:</h2>


<div dir="rtl">شرح ما يقوم به الكود (like code documentation)<div>

In [139]:
# your code here


In [140]:
# example test

<div dir="rtl">ملاحظاتك في حال وجودها</div>
<div dir="rtl">يمكنك إضافة خلايا لكل طلب بقدر ما تشاء، المهم أن تحافظ على تنظيم الملف</div>

# Question [1]: Load the data

## [1.1]

In [141]:
train_file_path = 'train.csv'
test_file_path = 'test.csv'
validation_file_path = 'val.csv'
train_data = pd.read_csv(train_file_path)
test_data = pd.read_csv(test_file_path)
validation_data = pd.read_csv(validation_file_path)

# print("number of data in train data : ",len(train_data))
# print("number of data in test data : ",len(test_data))
# print("number of data in validation data : ",len(validation_data))

In [142]:
# train_data.head(10)

In [143]:
# test_data.head(10)

In [144]:
# validation_data.head(10)

In [145]:
#drop_duplicates يزيل الصفوف المتكرره
#dropna يحذف الصفوف المكررة
test_data=test_data.drop_duplicates().dropna()
train_data=train_data.drop_duplicates().dropna()
validation_data=validation_data.drop_duplicates().dropna()
# print("number of data in train data after cleaning : ",len(train_data))
# print("number of data in test data after cleaning : ",len(test_data))
# print("number of data in validation data after cleaning : ",len(validation_data))

In [146]:
# train_data.head(10)

In [147]:
# test_data.head(10)

In [148]:
# validation_data.head(10)

## [1.2]

In [149]:
def unify_specialty(name):
    name = re.sub(r"[^\w\s]", " ", name)  # يزيل الرموز الترقيمية
    name = re.sub(r"\s+", " ", name).strip()  # يزيل الفراغات المتكرره #strip يزيل الفراغات يلي بالبداية والنهاية
    name =re.sub("[\_\-]"," ",name)
    return name

train_data['label']=train_data['label'].apply(unify_specialty)
test_data['label']=test_data['label'].apply(unify_specialty)
validation_data['label']=validation_data['label'].apply(unify_specialty)   


In [150]:
# x=set(train_data['label'])
# x

In [151]:
Dictionary_of_diseases= {
    'أمراض الجهاز التنفسي': 'أمراض الجهاز التنفسي',
    'الجهاز التنفسي': 'أمراض الجهاز التنفسي',
    'امراض الجهاز التنفسي': 'أمراض الجهاز التنفسي',
    'أمراض الدم': 'أمراض الدم',
    'الدم': 'أمراض الدم',
    'امراض الدم': 'أمراض الدم',
    'أمراض الغدد الصماء': 'أمراض الغدد الصماء',
    'الغدد الصماء': 'أمراض الغدد الصماء',
    'امراض الغدد الصماء': 'أمراض الغدد الصماء',
    'مرض السكري': 'مرض السكري',
    'السكري': 'مرض السكري',
    'الاورام الخبيثة والحميدة':'الأورام الخبيثة والحميدة'

}

def Unification_name_of_diseases(c):
    c=c.map(Dictionary_of_diseases).fillna(c)
    return c

In [152]:
train_data['label']=Unification_name_of_diseases(train_data['label'])
test_data['label']=Unification_name_of_diseases(test_data['label'])
validation_data['label']=Unification_name_of_diseases(validation_data['label'])

In [153]:
# x=set(train_data['label'])
# x

# Question [2]: Text Analysis

## [2.1] 


In [154]:
# train_data

In [155]:
# Train data

# عدد الأجوبة الخاصة بأمراض الدم :
answers_len_in_blood_diseases = len(train_data[train_data['label']=='أمراض الدم']['answer'].tolist())
# print(answers_len_in_blood_diseases)

# عدد الأسئلة الخاصة بأمراض الدم :
questions_len_in_blood_diseases = len(train_data[train_data['label']=='أمراض الدم']['question'].tolist())
# print(questions_len_in_blood_diseases)


# عدد الداتا الخاصة بأمراض الدم :
total_blood_diseases_len = len(train_data[train_data['label']=='أمراض الدم'])
print('عدد الداتا الخاصة بأمراض الدم:',total_blood_diseases_len)

# عدد الداتا في أمراض الجهاز التنفسي :
total_blood_diseases_len = len(train_data[train_data['label']=='أمراض الجهاز التنفسي'])
print('عدد الداتا الخاصة بأمراض الجهاز التنفسي:',total_blood_diseases_len)

# غدد الداتا في أمراض الغدد الصماء :
total_blood_diseases_len = len(train_data[train_data['label']=='أمراض الغدد الصماء'])
print('عدد الداتا الخاصة بأمراض الغدد الصماء:',total_blood_diseases_len)

# عدد  الداتا ارتفاع ضغط الدم :
total_blood_diseases_len = len(train_data[train_data['label']=='ارتفاع ضغط الدم'])
print('عدد الداتا الخاصة بأمراض ارتفاع ضغط الدم:',total_blood_diseases_len)

# عدد الداتا في الأورام الخبيثة والحميدة :
total_blood_diseases_len = len(train_data[train_data['label']=='الأورام الخبيثة والحميدة'])
print('عدد الداتا الخاصة بالأورام الخبيثة والحميدة:',total_blood_diseases_len)

# عدد الداتا في جراحة العظام
total_blood_diseases_len = len(train_data[train_data['label']=='جراحة العظام'])
print('عدد الداتا الخاصة بجراحةالعظام:',total_blood_diseases_len)

# عدد الداتا في الجراحة العامة
total_blood_diseases_len = len(train_data[train_data['label']=='جراحة عامة'])
print('عدد الداتا الخاصة بالجراحة العامة:',total_blood_diseases_len)

# عدد الداتا في مرض السكري
total_blood_diseases_len = len(train_data[train_data['label']=='مرض السكري'])
print('عدد الداتا الخاصة بمرض السكري:',total_blood_diseases_len)


عدد الداتا الخاصة بأمراض الدم: 1398
عدد الداتا الخاصة بأمراض الجهاز التنفسي: 3676
عدد الداتا الخاصة بأمراض الغدد الصماء: 5752
عدد الداتا الخاصة بأمراض ارتفاع ضغط الدم: 2537
عدد الداتا الخاصة بالأورام الخبيثة والحميدة: 6449
عدد الداتا الخاصة بجراحةالعظام: 2211
عدد الداتا الخاصة بالجراحة العامة: 6307
عدد الداتا الخاصة بمرض السكري: 4268


In [156]:
# # Drawing pie chart:
# # ملاحظة بالعربي طلع الخط مفشكل ف تمت الكتابة بالانكليزي



# data = {
#     'الاختصاص': ['أمراض الدم', 'أمراض الجهاز التنفسي', 'أمراض الغدد الصماء','أمراض ارتفاع ضغط الدم','الأمراض الخبيثة والحميدة','جراحةالعظام','الجراحة العامة','مرض السكري'],
#     'عدد النصوص': [2153, 6002, 9417,4190,10711,2641,10548,7096]
# }

# data


# df = pd.DataFrame(data)

# # print(df)
# arabic_font = FontProperties(fname='font/NotoNaskhArabic-VariableFont_wght.ttf') 

# df['الاختصاص'] = [get_display(arabic_reshaper.reshape(label)) for label in df['الاختصاص']]
# plt.figure(figsize=(8, 8))
# plt.pie(df['عدد النصوص'], labels=df['الاختصاص'], autopct='%1.1f%%', startangle=140)
# plt.title(get_display(arabic_reshaper.reshape('توزيع النصوص حسب الاختصاص')), fontproperties=arabic_font)
# plt.show()

# # import pandas as pd
# # import matplotlib.pyplot as plt

# # data = {
# #     'Specialization': ['Blood Diseases', 'Respiratory System Diseases', 'Endocrine Diseases', 'Hypertension', 'Malignant and Benign Diseases', 'Orthopedic Surgery', 'General Surgery', 'Diabetes'],
# #     'Number of Texts': [1398, 3676, 5752, 2537, 6449, 2211, 6307, 4268]
# # }

# # df = pd.DataFrame(data)

# # print(df)

# # plt.figure(figsize=(8, 8))
# # plt.pie(df['Number of Texts'], labels=df['Specialization'], autopct='%1.1f%%', startangle=140)
# # plt.title('Distribution of Texts by Specialization')
# # plt.show()


## [2.2] 

In [157]:
def delete_repeated_letter(text):
    # cleaned_text = re.sub(r'(.)\1{2,}', r'\1', text)
    cleaned_text = re.sub(r'([^\w\s\.])\1+', r'\1', text)
    cleaned_text = re.sub(r'(\.\s*)\1+$', r'\1', cleaned_text)
    return cleaned_text

def handle_connected_words(tokenized):
    # Handle connected words with '-'
    result_tokens = []
    for token in tokenized:
        # Split connected words with '-'
        result_tokens.extend(token.split('-'))
    return result_tokens


def filter_text(list):
    list_of_phrases =[]
    all_tokens = []
    for sentence in list:
        tokenizer = RegexpTokenizer(r'\b\d+\b|\b[^\d\W_]{2,}\b|[^\d\W_]+(?:-[^\d\W_]+)?|\S')
        tokenized = tokenizer.tokenize(delete_repeated_letter(sentence))
        tokenized = handle_connected_words(tokenized)

        # Remove '-' and '/' and one-letter words from each token
        # cleaned_tokens = [token for token in tokenized if len(token) > 1 and token not in ['-', '/']]
        # cleaned_tokens = [token if len(token) > 1 or token == 'و' else '' for token in tokenized if token not in ['-', '/']]

        cleaned_tokens = [token if (token.isalpha() and len(token) > 1) or token == 'و' else '' for token in tokenized if token not in ['-', '/']]

        combined_phrase = ' '.join(cleaned_tokens)
        list_of_phrases.append(combined_phrase)

    for phrase in list_of_phrases:
        tokens = nltk.word_tokenize(phrase)
        all_tokens.append(tokens)
        
    return all_tokens



In [158]:
train_data_answers_list = train_data['answer'].values
answers = filter_text(train_data_answers_list)
flattened_answers = [item for sublist in answers for item in sublist]
print(len(flattened_answers))


1645245


In [159]:
train_data_questions_list = train_data['question'].values
questions = filter_text(train_data_questions_list)
flattened_quesions = [item for sublist in questions for item in sublist]
print(len(flattened_quesions))

801979


In [160]:
# حساب عدد الكلمات الكلي :
total_words_len = len(flattened_quesions) + len(flattened_answers)
print(total_words_len)

2447224


In [161]:
# حساب عدد الكلمات الفريدة :
total_unique_words=len(set(flattened_quesions+flattened_answers))
print("عدد الكلمات الفريدة:", total_unique_words)

عدد الكلمات الفريدة: 94614


## [2.3] 

In [162]:
# WCD=dict()
# train_data_answers_list = flattened_answers
# train_data_questions_list = flattened_quesions
# train_data_list=train_data_answers_list+train_data_questions_list
# answers_text = ' '.join(train_data_list)
# reshaped_text = arabic_reshaper.reshape(answers_text)
# arabic_text = get_display(reshaped_text)
# wordcloud = WordCloud(font_path='font/NotoNaskhArabic-VariableFont_wght.ttf', background_color='white').generate(arabic_text)
# WCD=wordcloud.words_
# plt.figure(figsize=(50, 50))
# plt.imshow(wordcloud, interpolation='bilinear')
# plt.axis("off")
# plt.show()

## [2.4] 

In [163]:
def tokenized_text(text):
    # allWords = nltk.tokenize.word_tokenize(text)

    allWordDist = nltk.FreqDist(w for w in text)
    stopwords = nltk.corpus.stopwords.words('english')
    allWordExceptStopDist = nltk.FreqDist(w.lower() for w in text if w not in stopwords)  
    mostCommon= allWordDist.most_common(15)
    print('أكثر 15 كلمة مكررة:',mostCommon)
    leastCommon = allWordDist.most_common()[:-11:-1]
    print('أقل 10 كلمة مكررة:', leastCommon)
    bigram_freq = nltk.FreqDist(bigrams(text))
    most_common_bigrams = bigram_freq.most_common(10)
    print('أكثر 10 ثنائيات مكررة في النص:',most_common_bigrams)
    trigram_freq = nltk.FreqDist(trigrams(text))
    most_common_trigrams = trigram_freq.most_common(10)
    print('أكثر 10 ثلاثيات كلمات مكررة في النص:',most_common_trigrams)
    collocations_bigram = nltk.collocations.BigramCollocationFinder.from_words(text)
    most_common_collocations_bigram = collocations_bigram.nbest(nltk.collocations.BigramAssocMeasures.likelihood_ratio, 10)
    print('أكثر 10 ثنائيات كلمات مهمة في النص:',most_common_collocations_bigram)
    collocations_trigram = nltk.collocations.TrigramCollocationFinder.from_words(text)
    most_common_collocations_trigram = collocations_trigram.nbest(nltk.collocations.TrigramAssocMeasures.likelihood_ratio, 10)
    print('أكثر 10 ثلاثيات كلمات مهمة في النص:',most_common_collocations_trigram)  
    print('========================================================================================================================================================================================')  


    


# tokenized_text('tony botros is tony the tony best best best tony botros in in in the whole world tony botros')
# tokenized_text(['طوني','طوني','بطرس','هو','طوني','هو','بطرس','ال','الأفضل'])
# tokenized_text(['لدي', 'جرح', 'فوق', 'حاجبي', 'و', 'تمت', 'عملية', 'الخياطة', 'ثم', 'نزع', 'الغرزات', 'بشكل', 'عادي', 'لكن', 'مر', 'اسبوع', 'على', 'ذلك', 'و', 'مازال', 'هناك', 'الم', 'و', 'تطلب', 'شديد', 'في', 'المنطقة', 'لدرجة', 'اني', 'لا', 'استطيع', 'رفع', 'حاجبي', 'ابدا', 'إضافة', 'لوجود', 'انتفاخ', 'هل', 'هدا', 'طبيعي', 'ام', 'يمكن', 'ان', 'يكون', 'خلل', 'و', 'إن', 'كان', 'كذلك', 'ما', 'هو'])
# tokenized_text(answers[0])

In [164]:
#الشرح

In [165]:
# هاد التابع لمرق كل اختصاص وجيب منو الكلمات كلا متل مو مطلوب ب 2.4
def all_tokinized_text(specialized):
    filtered_questions = filter_text(train_data[train_data['label']==specialized]['question'].values)
    filtered_answers = filter_text(train_data[train_data['label']==specialized]['answer'].values)
    list = filtered_questions + filtered_answers
    flattened_list = [item for sublist in list for item in sublist]
    # flattened_list
    tokenized_text(flattened_list)
    

In [166]:
# all_tokinized_text('أمراض الدم')

In [167]:
# all_tokinized_text('أمراض الجهاز التنفسي')

In [168]:
# all_tokinized_text('أمراض الغدد الصماء')

In [169]:
# all_tokinized_text('ارتفاع ضغط الدم')

In [170]:
# all_tokinized_text('الأورام الخبيثة والحميدة')

In [171]:
# all_tokinized_text('جراحة العظام')

In [172]:
# all_tokinized_text('جراحة عامة')

In [173]:
# all_tokinized_text('مرض السكري')

In [174]:
# # على كل الداتا تبع الترين:
# filtered_questions = filter_text(train_data['question'].values)
# filtered_answers = filter_text(train_data['answer'].values)
# list = filtered_questions + filtered_answers
# flattened_list = [item for sublist in list for item in sublist]
# flattened_list
# tokenized_text(flattened_list)


In [175]:
#الشرح

## [2.5] 

In [176]:
train_data['all_word']=train_data['answer']+train_data['question']
train_data_list=train_data['all_word'].values
all_word = filter_text(train_data_list)

In [177]:
flattened_all_word = [item for sublist in all_word for item in sublist]
train_data['length_of_sentinse']=train_data['all_word'].apply(len)
# train_data['length_of_sentinse']

In [178]:
w=[]
for i in all_word:
    w.append(set(i))
train_data['unique_words']=w
train_data['number_of_unique_words']=train_data['unique_words'].apply(len)

In [179]:
def histogram(length_of_sentinse,number_of_unique_words):
    plt.figure(figsize=(10, 6))
    plt.subplot(2, 1, 1)
    # plt.bar(number_of_unique_words, length_of_sentinse, color='blue', alpha=0.7)
    plt.hist(length_of_sentinse, label='length_of_sentinse',edgecolor='black', linewidth=1.2,bins=30, alpha=0.5)
    plt.hist(number_of_unique_words, label='number_of_unique_words',edgecolor='black', linewidth=1.2,bins=30, alpha=0.5)
    plt.title('histogram')
    plt.xlabel('length_of_sentinse')
    plt.ylabel('number_of_unique_words')
    # plt.tight_layout()
    plt.legend()
    plt.show()

In [180]:
# لكل النص
# histogram(train_data['length_of_sentinse'],train_data['number_of_unique_words'])

In [181]:
# # أمراض الجهاز التنفسي
# histogram(train_data[train_data['label']=='أمراض الجهاز التنفسي']['length_of_sentinse'],train_data[train_data['label']=='أمراض الجهاز التنفسي']['number_of_unique_words'])

In [182]:
# # أمراض الدم
# histogram(train_data[train_data['label']=='أمراض الدم']['length_of_sentinse'],train_data[train_data['label']=='أمراض الدم']['number_of_unique_words'])

In [183]:
# # أمراض الغدد الصماء
# histogram(train_data[train_data['label']=='أمراض الغدد الصماء']['length_of_sentinse'],train_data[train_data['label']=='أمراض الغدد الصماء']['number_of_unique_words'])

In [184]:
# # ارتفاع ضغط الدم
# histogram(train_data[train_data['label']=='ارتفاع ضغط الدم']['length_of_sentinse'],train_data[train_data['label']=='ارتفاع ضغط الدم']['number_of_unique_words'])

In [185]:
# # الأورام الخبيثة والحميدة
# histogram(train_data[train_data['label']=='الأورام الخبيثة والحميدة']['length_of_sentinse'],train_data[train_data['label']=='الأورام الخبيثة والحميدة']['number_of_unique_words'])

In [186]:
# # جراحة العظام
# histogram(train_data[train_data['label']=='جراحة العظام']['length_of_sentinse'],train_data[train_data['label']=='جراحة العظام']['number_of_unique_words'])

In [187]:
# # جراحة عامة
# histogram(train_data[train_data['label']=='جراحة عامة']['length_of_sentinse'],train_data[train_data['label']=='جراحة عامة']['number_of_unique_words'])

In [188]:
# # مرض السكري
# histogram(train_data[train_data['label']=='مرض السكري']['length_of_sentinse'],train_data[train_data['label']=='مرض السكري']['number_of_unique_words'])

# Question [3]: Text Cleaning and Normalization

## [3.1] 


In [189]:
# train_data
# clean_tweet = re.sub(r'^RT(\s)+|https?\S+|#|@\S+', '', tweet) ## من أجل أزالت 
#r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
def remove_links(text):
    clean_text = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', text)
    return clean_text


In [190]:
# remove_links('https://chat.openai.com/c/dc21a08a-5be6-42e3-a6e4-0c1d26791a46 hi')

## [3.2] 

In [191]:
import re

def remove_doctors_prefix(text):
    pattern = r'\b(?:الدكتور(?:ة)?)\s+(?:الصيدلاني(?:ة)?)?\s*(?:([\u0600-\u06FF]+)\s*[\u0600-\u06FF]+\s*([\u0600-\u06FF]+))\b'

    def replacer(match):
        full_name = match.group(1)
        first_name = full_name.split()[0]
        return f'د.{first_name}'

    result = re.sub(pattern, replacer, text)

    return result


In [192]:
texts = train_data[train_data['label']=='أمراض الدم']['answer']
# texts.apply(remove_doctors_prefix)[0]
# for text in texts[:5]:
#     # print(text)
#     print(remove_doctors_prefix(text))
#     print("=================================================================")

## [3.3] 

In [193]:
def remove_sentences_after_doctor(text):
    result = re.sub(r'(الدكتور(?:ة)[^\n]*\n|د\.[^\n]*\n)([^\n]*\n){0,6}', r'\1', text)
    return result

# for text in texts[:5]:
#     print(remove_sentences_after_doctor(text))

In [194]:
def remove_sentences_after_doctor(text):
    text_without_empty_lines = re.sub(r'\n\s*\n', '\n', text)
    lines = text_without_empty_lines.splitlines()
    lines.pop()
    new_text = '\n'.join(lines)   
    return new_text

# for text in texts[:100]:
#     print(remove_sentences_after_doctor(text))
#     print("/////////////////////////////////////////////")

## [3.4] 

In [195]:
def remove_duplicate_letters_except_alif(text):
    pattern = re.compile(r'(?:(?<!ا)ا(?!ا)|(.)(?:\1)+)', re.UNICODE) # بشلي كلشي أحرف مكررة وخاصة الألف مابكررها اكتر من مرة
    result = pattern.sub(lambda x: x.group(1) if x.group(1) else 'ا', text)
    return result

# for text in texts[:5]:
#     print(remove_duplicate_letters_except_alif(text))

## [3.5]

### [3.5.1] 

In [196]:
# هاد بيقلب لل 1و2و3و4و....
def convert_numbers_to_arabic_v2(text):
    digit_mapping = {'٠': '0', '١': '1', '٢': '2', '٣': '3', '٤': '4', '٥': '5', '٦': '6', '٧': '7', '٨': '8', '٩': '9'}
    english_text = re.sub(r'[٠-٩]', lambda x: digit_mapping[x.group()], text)

    return english_text


# for text in texts[:5]:
#     print(convert_numbers_to_arabic_v2(text))

In [197]:
# هاد بيلقلب لل ١,٢,٣,٤
def convert_numbers_to_arabic(text):
    digit_mapping = {'0': '٠', '1': '١', '2': '٢', '3': '٣', '4': '٤', '5': '٥', '6': '٦', '7': '٧', '8': '٨', '9': '٩'}
    arabic_text = re.sub(r'[0-9]', lambda x: digit_mapping[x.group()], text)

    return arabic_text


# for text in texts[:5]:
#     print(convert_numbers_to_arabic(text))

### [3.5.2] 

In [198]:
def standardize_numbers(text, replacement_symbol='T'):
    digit_pattern = r'[0-9٠-٩]+'
    standardized_text = re.sub(digit_pattern, lambda x: replacement_symbol, text)

    return standardized_text


# for text in texts[:5]:
#     print(standardize_numbers(text))

### [3.5.3] 

In [199]:
def remove_numbers(text):
    digit_pattern = r'\d+'
    
    text_without_numbers = re.sub(digit_pattern, '', text)

    return text_without_numbers

# for text in texts[:5]:
#     print(remove_numbers(text))

### [3.5.4]

In [200]:
texts = train_data[train_data['label']=='أمراض الجهاز التنفسي']['answer'].values

def remove_dates_and_times(text):
    date_time_patterns = [
        r'\d{4}-\d{2}-\d{2}\s+\d{1,2}:\d{2}:\d{2}'  # Match dates and times like 2015-01-04 20:08:51
        # r'\d{1,2}/\d{1,2}/\d{2,4}',         # Match dates like 12/31/2022
        # r'\d{1,2}-\d{1,2}-\d{2,4}',         # Match dates like 12-31-2022
        # r'\d{1,2}:\d{2}:\d{2}'               # Match times like 12:34:56
        # r'\d{1,2}\s+(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\s*\d{2,4}',  # Match dates like 31 Dec 2022
        # r'\d{1,2}:\d{2}\s*(?:AM|PM|am|pm)?',  # Match times like 12:34 AM
    ]

    for pattern in date_time_patterns:
        text = re.sub(pattern, '', text)

    return text

# for text in texts[:5]:
#     print(remove_dates_and_times(text))

## [3.6] 

### [3.6.1] 

In [201]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [202]:
def remove_punctuation(text):
    text=re.sub(r'[!\"#$%&\'()*+,\-./:;<=>?@\[\\\]^_`{|}~،؛؟ـ]',' ',text)
    return text

# for text in texts[:5]:
#     print(remove_punctuation(text))

### [3.6.2] 

In [203]:
def keep_arabic_punctuations_numbers(text):
    text= re.sub(r'[^\u0600-\u06FF0-9،؛؟ـ!\"#$%&\'()*+,\-./:;<=>?@\[\\\]^_`{|}~]', ' ', text)
    return text

# for text in texts[:5]:
#     print(keep_arabic_punctuations_numbers(text))

## [3.7] 

In [204]:
stop_word=set(stop_words.classed_stopwords_list())
Some_stop_words_we_need=["قبلما","بعدما","فقط","مازال","ليست","ليس","ليسا","لسنا",
                         "لسن","لازالت","مساء","صباح","قبل","بعد","إياك","إياكن",
                         "إياكما","إياكم","لم","عدا","إلا","د","كلا","عامة","لا",
                         "حبذا","أقل","أكثر"]
Some_stop_words_dont_we_need=['عليكم','السلام','انا','أرجو','الرد','المزيد','إقرأ']
for word in Some_stop_words_we_need:
    if word in stop_word:
        stop_word.remove(word)

for word in Some_stop_words_we_need:
    if word is not stop_word:
        stop_word.add(word)

def remove_stop_words(text):
    text =re.sub("[\_\-\/]"," ",text)
    text=word_tokenize(text)
    text=[w for w in text if not w in stop_word]
    return text

# for text in texts[:5]:
#     print(remove_stop_words(text))

ملاحــــــــــــــــــظه


## [3.8] 

In [205]:
# def stem_arabic_words_farasa(text):
#     stemmer = FarasaStemmer()
#     text=stemmer.stem(text)
#     return text

# text = "الكتب في المكتبة متنوعة ومفيدة"
# stemmed_text = stem_arabic_words_farasa(text)
# print(stemmed_text)

In [206]:
# def stem_arabic_words_tashaphyne(text):
#     stemmer = ArabicLightStemmer()
#     words = text.split()
#     stemmed_words = [stemmer.light_stem(word) for word in words]
#     return ' '.join(stemmed_words)

# text = "الكتب في المكتبة متنوعة ومفيدة"
# stemmed_text = stem_arabic_words_tashaphyne(text)
# print(stemmed_text)

In [207]:
def stem_arabic_words_ISRIStemmer(text):
    stemmer = ISRIStemmer()
    words = text.split()
    stemmed_words = [stemmer.stem(word) for word in words]
    return ' '.join(stemmed_words)

# text = "الكتب في المكتبة متنوعة ومفيدة"
# stemmed_text = stem_arabic_words_ISRIStemmer(text)
# print(stemmed_text)

In [208]:
def stem_arabic_words_snowballstemmer(text):
    ar_stemmer = stemmer("arabic")
    words = text.split()
    stemmed_words = [ar_stemmer.stemWord(word) for word in words]
    return ' '.join(stemmed_words)

# text = "الكتب في المكتبة متنوعة ومفيدة"
# stemmed_text = stem_arabic_words_snowballstemmer(text)
# print(stemmed_text)

In [209]:
# for text in texts[:5]:
#     print(stem_arabic_words_snowballstemmer(text))

In [210]:
# for text in texts[:5]:
#     print(stem_arabic_words_tashaphyne(text))

In [211]:
# for text in texts[:5]:
#     print(stem_arabic_words_farasa(text))

In [212]:
# for text in texts[:5]:
#     print(stem_arabic_words_ISRIStemmer(text))

## [3.9] 

### [3.9.1] 

In [213]:
def unify_hamzat(text):
    unified_text = re.sub(r'[ءؤئ]', 'ء', text)
    return unified_text

In [214]:
# for text in texts[:5]:
#     print(unify_hamzat(text))

### [3.9.2] 

In [215]:
def unify_Alfat(text):
    unified_text = re.sub(r'[أإآ]', 'أ', text)
    return unified_text

In [216]:
# for text in texts[:5]:
#     print(unify_Alfat(text))

### [3.9.3] 

In [217]:
def remove_tatweel(text):
    text =re.sub( r'ـ', '', text)
    return text

In [218]:
# for text in texts[:5]:
#     print(remove_tatweel(text))

### [3.9.4] 

In [219]:
def remove_arabic_diacritics(text):
    text=re.sub(r'[\u064B-\u065F]', '', text)
    return text


In [220]:
# for text in texts[:5]:
#     print(remove_arabic_diacritics(text))

## [3.10] 

In [221]:
def remove_space(text):
    text = re.sub(r"\s+", " ", text).strip()
    return text

In [222]:
# for text in texts[:5]:
#     print(remove_space(text))

## Extra [3.11]

# Question [4]: Prepare Training Data

## [4.1] 


In [223]:
def preparing_training_data(text):
    text_after_cleaning_links = remove_links(text)
    text_after_removing_doctors_prefix = remove_doctors_prefix(text_after_cleaning_links)
    text_after_removing_specialization = remove_sentences_after_doctor(text_after_removing_doctors_prefix)
    text_after_removing_doublication_chars = remove_duplicate_letters_except_alif(text_after_removing_specialization)
    text_after_removing_date_and_time = remove_dates_and_times(text_after_removing_doublication_chars)
    text_after_removing_numbers = remove_numbers(text_after_removing_date_and_time)
    text_after_removing_non_arabic_words = keep_arabic_punctuations_numbers(text_after_removing_numbers)
    # text after removing stop words
    text_after_unify_hamzat = unify_hamzat(text_after_removing_non_arabic_words)
    text_after_unify_alafat = unify_Alfat(text_after_unify_hamzat)
    text_after_removing_tatwel = remove_tatweel(text_after_unify_alafat)
    text_after_removing_tashkel = remove_arabic_diacritics(text_after_removing_tatwel)
    text_after_removing_spaces_and_keeping_one_space = remove_space(text_after_removing_tashkel)
    # text after removing wrong arabic words and correct them


    return text_after_removing_spaces_and_keeping_one_space
    
    
    
questions = train_data['question'].values
answers = train_data['answer'].values

train_data['cleann_questions']=questions
train_data['cleann_answer']=answers
questions_answers = questions[0:5] + answers[0:5]

questions_answers.tolist()
# for i in questions_answers:
#     print(preparing_training_data(i))
    
    

['\r\nما هي مميزات و عيوب الدواء جلوكوفانس 500 5 و ايضا الأنسولين مكس تارد 30 \r\n\r\nلكل علاج ايجابيته وسلبياته والتي تعتمد على حالة المريض فما قد يناسب احدهم قد لا يناسب الاخر وهذا يحدده الطبيب خلال مراجعة المريض له\r\n1\r\n2015-01-04 20:08:51\r\n\r\n\r\nالدكتور يزن علي خليف\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n/اسئلة-طبية/مرض-السكري/ما-هي-مميزات-و-عيوب-الدواء-جلوكوفانس-و-ايضا-الانسولين-351157\r\n\r\nالدكتور يزن علي خليف \r\n\r\n\r\nالغدد الصماء \r\n\r\n\r\n',
 '\r\nاليك نتيجة تحليل هرمونات الغدة الدرقية علما بانه تم استأصال الغدة منذ اكثر من سنتينTT3=163TT4=12.6TSH=.01 هل مطلوب تعديل الجرعة \r\n\r\nنعم. يجب تخفيض الجرعة، الا اذا كان سبب استئصال الغدة هو سرطان الغدة\r\n0\r\n2017-02-13 06:34:19\r\n\r\n\r\nالدكتور باسم مرقص\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n/اسئلة-طبية/امراض-الغدد-الصماء/اليك-نتيجة-تحليل-هرمونات-الغدة-الدرقية-علما-بانه-تم-909154\r\n\r\nالدكتور باسم مرقص \r\n\r\n\r\nجراحة عامة \r\n\r\n\r\n',
 '\r\nحلول منزلية لأعراض ارتفاع ضغط الدم \r\n\r\nيفضل عدم الاستغناء عن العلاج الدوائي لم

## [4.2] 

In [224]:
all_train_data = questions + answers
all_train_data_After_cleaning = []
all_tokens = []
for line in all_train_data:
    training_data_after_cleaning = preparing_training_data(line)
    all_train_data_After_cleaning.append(training_data_after_cleaning)
    # print(training_data_after_cleaning)


# all_train_data_After_cleaning
# # print(tokens)
for phrase in all_train_data_After_cleaning:
    tokens = nltk.word_tokenize(phrase)
    all_tokens.append(tokens)   




flattened_all_tokens = [item for sublist in all_tokens for item in sublist]
print(len(flattened_all_tokens)) 

1939864


In [225]:
# حساب عدد الكلمات الفريدة :
total_unique_words=len(set(flattened_all_tokens))
print("عدد الكلمات الفريدة:", total_unique_words)

عدد الكلمات الفريدة: 128022


In [226]:
# نلاحظ ان عدد الكلمات الكلية قد قلت هنا أما عدد الكلمات المميزة قد زاد

## [4.3] 

In [227]:

df = pd.DataFrame(all_train_data_After_cleaning)

df = df.map(lambda x: '' if isinstance(x, (float, int)) else x.strip() if isinstance(x, str) and len(x) >= 5 else None)

df = df.dropna()
print(len(df))
print(len(all_train_data_After_cleaning))
# all_train_data_After_cleaning


32598
32598


# Question [5]: Baseline

The Comparison Dictionary example

In [228]:
# from keras.backend import clear_session
# clear_session()

In [229]:
comparison_table = {}

comparison_table['question_step_number'] = []
comparison_table['model_name'] = []
comparison_table['features'] = []
comparison_table['model_parameters'] = []
comparison_table['preprocessing_methods'] = []
comparison_table['accuracy'] = []
comparison_table['balance_accuracy'] = []

Filling the dictionary example

In [230]:
# comparison_table['question_step_number'].append("3.5")
# comparison_table['model_name'].append('logistic_regression')
# comparison_table['features'].append('bag of words')
# comparison_table['model_parameters'].append('default')
# comparison_table['preprocessing_methods'].append("remove links")
# comparison_table['accuracy'].append(accuracy)
# comparison_table['balance_accuracy'].append(balance_accuracy_value)

## [5.1]

In [231]:
def Prepare_data():
    X_train ,y_train,X_test,y_test,X_validation,y_validation=Prepare()
    X_train_combined = X_train['question'] + ' ' + X_train['answer']
    X_test_combined = X_test['question'] + ' ' + X_test['answer']
    X_validation_combined = X_validation['question'] + ' ' + X_validation['answer']
    return X_train_combined,X_test_combined,X_validation_combined

def Prepare():
    X_train, y_train = train_data[['question', 'answer']], train_data['label']
    X_test, y_test = test_data[['question', 'answer']], test_data['label']
    X_validation, y_validation = validation_data[['question', 'answer']], validation_data['label']
    return X_train ,y_train,X_test,y_test,X_validation,y_validation



In [232]:
# X_train, y_train = train_data[['question', 'answer']], train_data['label']
# X_test, y_test = test_data[['question', 'answer']], test_data['label']
# X_validation, y_validation = validation_data[['question', 'answer']], validation_data['label']


## [5.2]

In [233]:
X_train_combined ,X_test_combined,X_validation_combined = Prepare_data()
X_train ,y_train,X_test,y_test,X_validation,y_validation =Prepare()


def logistic_regression(x_train,x_test,x_val):

    vectorizer = CountVectorizer()

    X_train_bow = vectorizer.fit_transform(x_train)
    X_test_bow = vectorizer.transform(x_test)
    X_validation_bow = vectorizer.transform(x_val)

    logreg_model = LogisticRegression(max_iter=1000)

    logreg_model.fit(X_train_bow, y_train)

    y_test_pred = logreg_model.predict(X_test_bow)
    y_validation_pred = logreg_model.predict(X_validation_bow)

    accuracy_test = accuracy_score(y_test, y_test_pred)
    accuracy_validation = accuracy_score(y_validation, y_validation_pred)
    balanced_accuracy_test = balanced_accuracy_score(y_test, y_test_pred)


    print(f"Test Accuracy: {accuracy_test}")
    print(f"Validation Accuracy: {accuracy_validation}")
    print(f"Balanced Accuracy: {balanced_accuracy_test}")


    return accuracy_test, accuracy_validation, balanced_accuracy_test


In [234]:
# accuracy_test, accuracy_validation, balanced_accuracy_test = logistic_regression(X_train_combined,X_test_combined,X_validation_combined)

# comparison_table['question_step_number'].append("5.2")
# comparison_table['model_name'].append('logistic_regression')
# comparison_table['features'].append('bag of words')
# comparison_table['model_parameters'].append('default')
# comparison_table['preprocessing_methods'].append("nothing")
# comparison_table['accuracy'].append(accuracy_test)
# comparison_table['balance_accuracy'].append(balanced_accuracy_test)


ملاحظااااااااااااااااااااااااااااااااااات




نلاحظ ان ال test accuracy دقتو أكثر من ال balanced accuracy بنسبة بسيطة


## [5.3]

In [235]:
# # تنظيف النص من الروابط

# X_train_combined = X_train['question'].apply(remove_links) + ' ' + X_train['answer'].apply(remove_links)


# # print(len(X_train_without_links))
# print(len(X_train_combined))

# accuracy_test, accuracy_validation, balanced_accuracy_test = logistic_regression(X_train_combined,X_test_combined,X_validation_combined)

# comparison_table['question_step_number'].append("5.3")
# comparison_table['model_name'].append('logistic_regression')
# comparison_table['features'].append('bag of words')
# comparison_table['model_parameters'].append('default')
# comparison_table['preprocessing_methods'].append("Removing Links")
# comparison_table['accuracy'].append(accuracy_test)
# comparison_table['balance_accuracy'].append(balanced_accuracy_test)

 

In [236]:
# # حذف اسماء الأطباء مع اللقب
# X_train_combined = X_train['question'].apply(remove_doctors_prefix) + ' ' + X_train['answer'].apply(remove_doctors_prefix)


# # print(len(X_train_without_links))
# print(len(X_train_combined))

# accuracy_test, accuracy_validation, balanced_accuracy_test = logistic_regression(X_train_combined,X_test_combined,X_validation_combined)

# comparison_table['question_step_number'].append("5.3")
# comparison_table['model_name'].append('logistic_regression')
# comparison_table['features'].append('bag of words')
# comparison_table['model_parameters'].append('default')
# comparison_table['preprocessing_methods'].append("ٌRemoving doctors prefix")
# comparison_table['accuracy'].append(accuracy_test)
# comparison_table['balance_accuracy'].append(balanced_accuracy_test)



In [237]:
# # حذف المحارف المكررة من النص
# X_train_combined = X_train['question'].apply(remove_duplicate_letters_except_alif) + ' ' + X_train['answer'].apply(remove_duplicate_letters_except_alif)


# # print(len(X_train_without_links))
# print(len(X_train_combined))

# accuracy_test, accuracy_validation, balanced_accuracy_test = logistic_regression(X_train_combined,X_test_combined,X_validation_combined)

# comparison_table['question_step_number'].append("5.3")
# comparison_table['model_name'].append('logistic_regression')
# comparison_table['features'].append('bag of words')
# comparison_table['model_parameters'].append('default')
# comparison_table['preprocessing_methods'].append("Removing duplicate letters except alif")
# comparison_table['accuracy'].append(accuracy_test)
# comparison_table['balance_accuracy'].append(balanced_accuracy_test)



In [238]:
# # تحويل الارقام الى العربية
# X_train_combined = X_train['question'].apply(convert_numbers_to_arabic_v2) + ' ' + X_train['answer'].apply(convert_numbers_to_arabic_v2)


# # print(len(X_train_without_links))
# print(len(X_train_combined))

# accuracy_test, accuracy_validation, balanced_accuracy_test = logistic_regression(X_train_combined,X_test_combined,X_validation_combined)

# comparison_table['question_step_number'].append("5.3")
# comparison_table['model_name'].append('logistic_regression')
# comparison_table['features'].append('bag of words')
# comparison_table['model_parameters'].append('default')
# comparison_table['preprocessing_methods'].append("Removing Links")
# comparison_table['accuracy'].append(accuracy_test)
# comparison_table['balance_accuracy'].append(balanced_accuracy_test)



In [239]:
# # تحويل الارقام الى رمز من اختياري
# X_train_combined = X_train['question'].apply(standardize_numbers) + ' ' + X_train['answer'].apply(standardize_numbers)


# # print(len(X_train_without_links))
# print(len(X_train_combined))

# accuracy_test, accuracy_validation, balanced_accuracy_test = logistic_regression(X_train_combined,X_test_combined,X_validation_combined)

# comparison_table['question_step_number'].append("5.3")
# comparison_table['model_name'].append('logistic_regression')
# comparison_table['features'].append('bag of words')
# comparison_table['model_parameters'].append('default')
# comparison_table['preprocessing_methods'].append("standardize numbers to specific char")
# comparison_table['accuracy'].append(accuracy_test)
# comparison_table['balance_accuracy'].append(balanced_accuracy_test)



In [240]:
# # حذف الأرقام كلها
# X_train_combined = X_train['question'].apply(remove_numbers) + ' ' + X_train['answer'].apply(remove_numbers)


# # print(len(X_train_without_links))
# print(len(X_train_combined))

# accuracy_test, accuracy_validation, balanced_accuracy_test = logistic_regression(X_train_combined,X_test_combined,X_validation_combined)

# comparison_table['question_step_number'].append("5.3")
# comparison_table['model_name'].append('logistic_regression')
# comparison_table['features'].append('bag of words')
# comparison_table['model_parameters'].append('default')
# comparison_table['preprocessing_methods'].append("Removing numbers")
# comparison_table['accuracy'].append(accuracy_test)
# comparison_table['balance_accuracy'].append(balanced_accuracy_test)



In [241]:
# # حذف التاريخ والوقت
# X_train_combined = X_train['question'].apply(remove_dates_and_times) + ' ' + X_train['answer'].apply(remove_dates_and_times)


# # print(len(X_train_without_links))
# print(len(X_train_combined))

# accuracy_test, accuracy_validation, balanced_accuracy_test = logistic_regression(X_train_combined,X_test_combined,X_validation_combined)

# comparison_table['question_step_number'].append("5.3")
# comparison_table['model_name'].append('logistic_regression')
# comparison_table['features'].append('bag of words')
# comparison_table['model_parameters'].append('default')
# comparison_table['preprocessing_methods'].append("Removing Date and time")
# comparison_table['accuracy'].append(accuracy_test)
# comparison_table['balance_accuracy'].append(balanced_accuracy_test)



In [242]:
# # حذف المحارف الغير عربية
# X_train_combined = X_train['question'].apply(keep_arabic_punctuations_numbers) + ' ' + X_train['answer'].apply(keep_arabic_punctuations_numbers)


# # print(len(X_train_without_links))
# print(len(X_train_combined))

# accuracy_test, accuracy_validation, balanced_accuracy_test = logistic_regression(X_train_combined,X_test_combined,X_validation_combined)

# comparison_table['question_step_number'].append("5.3")
# comparison_table['model_name'].append('logistic_regression')
# comparison_table['features'].append('bag of words')
# comparison_table['model_parameters'].append('default')
# comparison_table['preprocessing_methods'].append("Remove non arabic words")
# comparison_table['accuracy'].append(accuracy_test)
# comparison_table['balance_accuracy'].append(balanced_accuracy_test)



In [243]:
# # حذف كلمات التوقف
# X_train_combined = X_train['question'].apply(lambda x: ' '.join(remove_stop_words(x))) + ' ' + X_train['answer'].apply(lambda x: ' '.join(remove_stop_words(x)))

# # print(len(X_train_without_links))
# print(len(X_train_combined))

# accuracy_test, accuracy_validation, balanced_accuracy_test = logistic_regression(X_train_combined,X_test_combined,X_validation_combined)

# comparison_table['question_step_number'].append("5.3")
# comparison_table['model_name'].append('logistic_regression')
# comparison_table['features'].append('bag of words')
# comparison_table['model_parameters'].append('default')
# comparison_table['preprocessing_methods'].append("Remove non arabic words")
# comparison_table['accuracy'].append(accuracy_test)
# comparison_table['balance_accuracy'].append(balanced_accuracy_test)



In [244]:
# # تجذير الكلمات العربية
# X_train_combined = X_train['question'].apply(stem_arabic_words_snowballstemmer) + ' ' + X_train['answer'].apply(stem_arabic_words_snowballstemmer) # 0,55 time : 2:50 m
# # X_train_combined = X_train['question'].apply(stem_arabic_words_tashaphyne) + ' ' + X_train['answer'].apply(stem_arabic_words_tashaphyne) # 0,54 time : 5:26 m
# # X_train_combined = X_train['question'].apply(stem_arabic_words_farasa) + ' ' + X_train['answer'].apply(stem_arabic_words_farasa) # **** time more than 35:00 m and he stills running
# # X_train_combined = X_train['question'].apply(stem_arabic_words_ISRIStemmer) + ' ' + X_train['answer'].apply(stem_arabic_words_ISRIStemmer) # 0.53 time 2:30 m 

# stem_arabic_words_ISRIStemmer
# # print(len(X_train_without_links))
# print(len(X_train_combined))

# accuracy_test, accuracy_validation, balanced_accuracy_test = logistic_regression(X_train_combined,X_test_combined,X_validation_combined)

# comparison_table['question_step_number'].append("5.3")
# comparison_table['model_name'].append('logistic_regression')
# comparison_table['features'].append('bag of words')
# comparison_table['model_parameters'].append('default')
# comparison_table['preprocessing_methods'].append("stem arabic words")
# comparison_table['accuracy'].append(accuracy_test)
# comparison_table['balance_accuracy'].append(balanced_accuracy_test)



In [245]:
# # توحيد طريقة كتابة الهمزات
# X_train_combined = X_train['question'].apply(unify_hamzat) + ' ' + X_train['answer'].apply(unify_hamzat)


# # print(len(X_train_without_links))
# print(len(X_train_combined))

# accuracy_test, accuracy_validation, balanced_accuracy_test = logistic_regression(X_train_combined,X_test_combined,X_validation_combined)

# comparison_table['question_step_number'].append("5.3")
# comparison_table['model_name'].append('logistic_regression')
# comparison_table['features'].append('bag of words')
# comparison_table['model_parameters'].append('default')
# comparison_table['preprocessing_methods'].append("unify hamzat")
# comparison_table['accuracy'].append(accuracy_test)
# comparison_table['balance_accuracy'].append(balanced_accuracy_test)



In [246]:
# # توحيد طريقة كتابة الألفات
# X_train_combined = X_train['question'].apply(unify_Alfat) + ' ' + X_train['answer'].apply(unify_Alfat)


# # print(len(X_train_without_links))
# print(len(X_train_combined))

# accuracy_test, accuracy_validation, balanced_accuracy_test = logistic_regression(X_train_combined,X_test_combined,X_validation_combined)

# comparison_table['question_step_number'].append("5.3")
# comparison_table['model_name'].append('logistic_regression')
# comparison_table['features'].append('bag of words')
# comparison_table['model_parameters'].append('default')
# comparison_table['preprocessing_methods'].append("unify Alafat")
# comparison_table['accuracy'].append(accuracy_test)
# comparison_table['balance_accuracy'].append(balanced_accuracy_test)



In [247]:
# # حذف التطويل
# X_train_combined = X_train['question'].apply(remove_tatweel) + ' ' + X_train['answer'].apply(remove_tatweel)


# # print(len(X_train_without_links))
# print(len(X_train_combined))

# accuracy_test, accuracy_validation, balanced_accuracy_test = logistic_regression(X_train_combined,X_test_combined,X_validation_combined)

# comparison_table['question_step_number'].append("5.3")
# comparison_table['model_name'].append('logistic_regression')
# comparison_table['features'].append('bag of words')
# comparison_table['model_parameters'].append('default')
# comparison_table['preprocessing_methods'].append("remove tatweel")
# comparison_table['accuracy'].append(accuracy_test)
# comparison_table['balance_accuracy'].append(balanced_accuracy_test)



In [248]:
# # حذف التشكيل
# X_train_combined = X_train['question'].apply(remove_arabic_diacritics) + ' ' + X_train['answer'].apply(remove_arabic_diacritics)


# # print(len(X_train_without_links))
# print(len(X_train_combined))

# accuracy_test, accuracy_validation, balanced_accuracy_test = logistic_regression(X_train_combined,X_test_combined,X_validation_combined)

# comparison_table['question_step_number'].append("5.3")
# comparison_table['model_name'].append('logistic_regression')
# comparison_table['features'].append('bag of words')
# comparison_table['model_parameters'].append('default')
# comparison_table['preprocessing_methods'].append("remove arabic diacritics")
# comparison_table['accuracy'].append(accuracy_test)
# comparison_table['balance_accuracy'].append(balanced_accuracy_test)



In [249]:
# # حذف الفراغات
# X_train_combined = X_train['question'].apply(remove_space) + ' ' + X_train['answer'].apply(remove_space)


# # print(len(X_train_without_links))
# print(len(X_train_combined))

# accuracy_test, accuracy_validation, balanced_accuracy_test = logistic_regression(X_train_combined,X_test_combined,X_validation_combined)

# comparison_table['question_step_number'].append("5.3")
# comparison_table['model_name'].append('logistic_regression')
# comparison_table['features'].append('bag of words')
# comparison_table['model_parameters'].append('default')
# comparison_table['preprocessing_methods'].append("remove spaces with one space")
# comparison_table['accuracy'].append(accuracy_test)
# comparison_table['balance_accuracy'].append(balanced_accuracy_test)



In [250]:
# # حذف اختصاص الطبيب
# X_train_combined = X_train['question'].apply(remove_sentences_after_doctor) + ' ' + X_train['answer'].apply(remove_sentences_after_doctor)


# # print(len(X_train_without_links))
# print(len(X_train_combined))

# accuracy_test, accuracy_validation, balanced_accuracy_test = logistic_regression(X_train_combined,X_test_combined,X_validation_combined)

# comparison_table['question_step_number'].append("5.3")
# comparison_table['model_name'].append('logistic_regression')
# comparison_table['features'].append('bag of words')
# comparison_table['model_parameters'].append('default')
# comparison_table['preprocessing_methods'].append("remove doctore specialization")
# comparison_table['accuracy'].append(accuracy_test)
# comparison_table['balance_accuracy'].append(balanced_accuracy_test)



## [5.4]

In [251]:
# 0.6437688353128878
def callfunction(text):
    # text=text.apply(remove_links) # 0.6434142882467647
    # text=text.apply(remove_doctors_prefix) # 0.639691544052473
    # text=text.apply(remove_duplicate_letters_except_alif) # 0.6353483424924659
    # text=text.apply(convert_numbers_to_arabic_v2) # 0.6424392838149264
    # text=text.apply(standardize_numbers) # 0.6348165218932813
    # text=text.apply(remove_numbers) # 0.6348165218932813
    # text=text.apply(remove_dates_and_times) # 0.6217869172132601
    # text=text.apply(keep_arabic_punctuations_numbers) # 0.6435915617798262
    # text=text.apply(lambda x: ' '.join(remove_stop_words(x))) # 0.6085800390001773
    # text=text.apply(stem_arabic_words_snowballstemmer) # 0.553270696684985
    # text=text.apply(unify_hamzat) # 0.6403120014181882
    text=text.apply(unify_Alfat) # 0.6448324765112569  # good
    text=text.apply(remove_tatweel) # 0.6437688353128878 # same 
    # text=text.apply(remove_arabic_diacritics) # 0.6422620102818649
    text=text.apply(remove_space) # 0.6437688353128878 # same
    # text=text.apply(remove_sentences_after_doctor) # 0.6217869172132601
    
    return text


In [252]:
# X_train_combined = X_train['question'] + ' ' + X_train['answer']
# X_train_combined = callfunction(X_train_combined)

In [253]:
# accuracy_test, accuracy_validation, balanced_accuracy_test = logistic_regression(X_train_combined,X_test_combined,X_validation_combined)

# comparison_table['question_step_number'].append("5.4")
# comparison_table['model_name'].append('logistic_regression')
# comparison_table['features'].append('bag of words')
# comparison_table['model_parameters'].append('default')
# comparison_table['preprocessing_methods'].append("callfunction")
# comparison_table['accuracy'].append(accuracy_test)
# comparison_table['balance_accuracy'].append(balanced_accuracy_test)

## [5.5]

In [254]:
# X_train_combined ,X_test_combined,X_validation_combined =Prepare_data()

# X_train_combined=callfunction(X_train_combined)
# # X_test_combined=callfunction(X_test_combined)

In [255]:
# tfidf_vectorizer  = TfidfVectorizer()
# train_data_X_train_combined = np.array(X_train_combined)
# train_data_X_test_combined = np.array(X_test_combined)
# train_data_X_validation_combined = np.array(X_validation_combined)
# tfidf_matrix_X_train_combined = tfidf_vectorizer .fit_transform(train_data_X_train_combined)
# tfidf_matrix_X_test_combined = tfidf_vectorizer .transform(train_data_X_test_combined)
# tfidf_matrix_X_validation_combined = tfidf_vectorizer .transform(train_data_X_validation_combined)


In [256]:
# from keras.backend import clear_session
# clear_session()

In [257]:
# # accuracy_test, accuracy_validation, balanced_accuracy_test = logistic_regression(tfidf_matrix_X_train_combined,tfidf_matrix_X_test_combined,tfidf_matrix_X_validation_combined)
# # model = make_pipeline(
# #     StandardScaler(with_mean=False),
# #     LogisticRegression(max_iter=1000, solver='lbfgs')
# # )


# model = LogisticRegression(max_iter=1000)

# model.fit(tfidf_matrix_X_train_combined, y_train)
# # predict_proba
# y_test_pred = model.predict(tfidf_matrix_X_test_combined)
# y_validation_pred = model.predict(tfidf_matrix_X_validation_combined)

# accuracy_test = accuracy_score(y_test, y_test_pred)
# accuracy_validation = accuracy_score(y_validation, y_validation_pred)
# balanced_accuracy_test = balanced_accuracy_score(y_test, y_test_pred)


# print(f"Test Accuracy: {accuracy_test}")
# print(f"Validation Accuracy: {accuracy_validation}")
# print(f"Balanced Accuracy: {balanced_accuracy_test}")


In [258]:
# comparison_table['question_step_number'].append("5.5")
# comparison_table['model_name'].append('logistic_regression')
# comparison_table['features'].append('TF-IDF')
# comparison_table['model_parameters'].append('default')
# comparison_table['preprocessing_methods'].append("callfunction")
# comparison_table['accuracy'].append(accuracy_test)
# comparison_table['balance_accuracy'].append(balanced_accuracy_test)

## [5.6]

In [259]:
# vectorizer  = TfidfVectorizer(min_df=2,max_df=0.5,ngram_range=(1,2))

# train_data_X_train_combined = np.array(X_train_combined)
# train_data_X_test_combined = np.array(X_test_combined)
# train_data_X_validation_combined = np.array(X_validation_combined)
# tfidf_matrix_X_train_combined = vectorizer .fit_transform(train_data_X_train_combined)
# tfidf_matrix_X_test_combined = vectorizer .transform(train_data_X_test_combined)
# tfidf_matrix_X_validation_combined = vectorizer .transform(train_data_X_validation_combined)


In [260]:
# model = LogisticRegression(max_iter=1000)

# model.fit(tfidf_matrix_X_train_combined, y_train)
# # predict_proba
# y_test_pred = model.predict(tfidf_matrix_X_test_combined)
# y_validation_pred = model.predict(tfidf_matrix_X_validation_combined)

# accuracy_test = accuracy_score(y_test, y_test_pred)
# accuracy_validation = accuracy_score(y_validation, y_validation_pred)
# balanced_accuracy_test = balanced_accuracy_score(y_test, y_test_pred)


# print(f"Test Accuracy: {accuracy_test}")
# print(f"Validation Accuracy: {accuracy_validation}")
# print(f"Balanced Accuracy: {balanced_accuracy_test}")

In [261]:
# comparison_table['question_step_number'].append("5.6")
# comparison_table['model_name'].append('logistic_regression')
# comparison_table['features'].append('TF-IDF with set transactions')
# comparison_table['model_parameters'].append('default')
# comparison_table['preprocessing_methods'].append("callfunction")
# comparison_table['accuracy'].append(accuracy_test)
# comparison_table['balance_accuracy'].append(balanced_accuracy_test)

# Question [6]: Semantic Representation

## [6.1]

In [262]:
# from sklearn.feature_extraction.text import CountVectorizer

# X_train_combined = X_train['question'] + ' ' + X_train['answer']


# vectorizer = CountVectorizer()

# train_data_bow = vectorizer.fit_transform(X_train_combined)

# def get_word_occurrences(word, bow_matrix, texts):
#     # لحتى جيب الانديكس للكلمة من ضمن مصفوفة الباغ اوف ووردز
#     word_index = vectorizer.vocabulary_.get(word)

#     if word_index is not None:
#         # امشيلي عالنص واطبعلي الكلمة وكم كرة مكررة
#         for i, text in enumerate(texts):
#             occurrences = bow_matrix[i, word_index]
#             print(f"Text {i + 1}: Word '{word}' occurs {occurrences} times")
#     else:
#         print(f"The word '{word}' is not found in the bag-of-words matrix.")

# # Example usage
# # Assuming 'مميزات' is the word you want to check
# get_word_occurrences('حرارة', train_data_bow, X_train_combined)


## [6.2]

In [263]:
# def get_context_words(target_word, bow_matrix, texts, context_size=10):
#     word_index = vectorizer.vocabulary_.get(target_word)

#     if word_index is not None:
#         context_words = set()  

#         for i, text in enumerate(texts):
#             occurrences = bow_matrix[i, word_index]

#             if occurrences > 0: # اذا تكررت الكلمة مرة أو أكثر
#                 # جبلنا الكونتيكست ووردز ضمن ال الكونتيكست سايز
#                 start_index = max(0, word_index - context_size)
#                 end_index = min(len(vectorizer.get_feature_names_out()), word_index + context_size + 1)
#                 context_words.update(vectorizer.get_feature_names_out()[start_index:end_index])

#         return context_words
#     else:
#         print(f"The word '{target_word}' is not found in the bag-of-words matrix.")
#         return None

# target_word = 'حرارة'
# context_words = get_context_words(target_word, train_data_bow, X_train_combined)

# if context_words is not None:
#     print(f"Context words for '{target_word}': {context_words}")


## [6.3]

## [Notes]

عادة لا يتم استخدام تجذيع او تجذير الكلمات لانها تؤدي الى فقدان معلومات مهمة والدقيقة حول الكلمات -
لا حاجة لعمليات تقطيع اخرى -
في معظم الحالات، لا تحتاج جميع الكلمات إلى أن تمتلك أشعة داللية (word embeddings). يمكن ترك الكلمات النادرة أو التي ليس لها تأثير كبير على النموذج بدون تمثيل. هذا يقلل من حجم مصفوفة الأشعة الداللية ويحسن كفاءة التدريب -

In [302]:
# import tensorflow as tf
# import pandas as pd
# import tensorflow as tf
# from tensorflow.keras import layers
# import numpy as np
# import tqdm
# import re

# # Assuming you have already defined X_train as mentioned in your question
# X_train_combined = X_train['question'] + ' ' + X_train['answer']

# # Tokenize the text in X_train
# tokens_list = [list(sentence.lower().split()) for sentence in X_train]
# tokens = [token for tokens in tokens_list for token in tokens]

# # Build vocabulary
# vocab, index = {}, 1  # start indexing from 1
# vocab['<pad>'] = 0  # add a padding token
# for token in tokens:
#     if token not in vocab:
#         vocab[token] = index
#         index += 1
# vocab_size = len(vocab)

# # Print vocabulary
# print("Vocabulary:")
# print(vocab)

# # Create inverse vocabulary
# inverse_vocab = {index: token for token, index in vocab.items()}

# # Print inverse vocabulary
# print("\nInverse Vocabulary:")
# print(inverse_vocab)

# # Convert text to sequences
# sequences = [[vocab[word] for word in tokens] for tokens in tokens_list]

# # Print example sequence
# print("\nExample Sequence:")
# print(sequences[0])

# # Generate positive skip-grams
# window_size = 2
# positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
#     sequences[0],
#     vocabulary_size=vocab_size,
#     window_size=window_size,
#     negative_samples=0
# )

# # Print positive skip-grams
# print("\nPositive Skip-Grams:")
# for target, context in positive_skip_grams[:5]:
#     print(f"({target}, {context}): ({inverse_vocab[target]}, {inverse_vocab[context]})")





# # Generates skip-gram pairs with negative sampling for a list of sequences
# # (int-encoded sentences) based on window size, number of negative samples
# # and vocabulary size.
# # Generates skip-gram pairs with negative sampling for a list of sequences
# # (int-encoded sentences) based on window size, number of negative samples
# # and vocabulary size.
# def generate_training_data(sequences, window_size, num_ns, vocab_size, seed):
#   # Elements of each training example are appended to these lists.
#     targets, contexts, labels = [], [], []

#     # Build the sampling table for `vocab_size` tokens.
#     sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(vocab_size)

#     # Iterate over all sequences (sentences) in the dataset.
#     for sequence in tqdm.tqdm(sequences):

#         # Generate positive skip-gram pairs for a sequence (sentence).
#         positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
#             sequence,
#             vocabulary_size=vocab_size,
#             sampling_table=sampling_table,
#             window_size=window_size,
#             negative_samples=0)

#         # Iterate over each positive skip-gram pair to produce training examples
#         # with a positive context word and negative samples.
#         for target_word, context_word in positive_skip_grams:
#         context_class = tf.expand_dims(
#             tf.constant([context_word], dtype="int64"), 1)
#         negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
#             true_classes=context_class,
#             num_true=1,
#             num_sampled=num_ns,
#             unique=True,
#             range_max=vocab_size,
#             seed=seed,
#             name="negative_sampling")

#         # Build context and label vectors (for one target word)
#         context = tf.concat([tf.squeeze(context_class,1), negative_sampling_candidates], 0)
#         label = tf.constant([1] + [0]*num_ns, dtype="int64")

#         # Append each element from the training example to global lists.
#         targets.append(target_word)
#         contexts.append(context)
#         labels.append(label)

#     return targets, contexts, labels

# # Assuming you have already loaded your dataset and X_train, as you mentioned
# text_ds = tf.data.Dataset.from_tensor_slices(X_train).filter(lambda x: tf.cast(tf.strings.length(x), bool))

# # ... (unchanged)

# # Vectorize the data in text_ds.
# text_vector_ds = text_ds.batch(1024).prefetch(tf.data.AUTOTUNE).map(vectorize_layer).unbatch()

# sequences = list(text_vector_ds.as_numpy_iterator())

# # ... (unchanged)

# targets, contexts, labels = generate_training_data(
#     sequences=sequences,
#     window_size=2,
#     num_ns=4,
#     vocab_size=vocab_size,
#     seed=SEED)

# # ... (unchanged)

# # Rest of the code remains the same

# BATCH_SIZE = 1024
# BUFFER_SIZE = 10000
# dataset = tf.data.Dataset.from_tensor_slices(((targets, contexts), labels))
# dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

# dataset = dataset.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
# print(dataset)



Vocabulary:
{'<pad>': 0, 'question': 1, 'answer': 2}

Inverse Vocabulary:
{0: '<pad>', 1: 'question', 2: 'answer'}

Example Sequence:
[1]

Positive Skip-Grams:


In [264]:
from keras.models import Model
from keras.layers import Embedding, Dense, Reshape, Input, Dot
from keras.preprocessing.text import Tokenizer
from keras.backend import clear_session
from keras.preprocessing.sequence import skipgrams
import numpy as np

def build_skipgram_model(vocab_size, embedding_dim):
    input_target = Input((1,))
    input_context = Input((1,))
    
    embedding = Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=1)
    target = embedding(input_target)
    context = embedding(input_context)
    
    dot_product = Dot(axes=2)([target, context])
    dot_product = Reshape((1,), input_shape=(1, 1))(dot_product)
    
    output = Dense(vocab_size, activation='softmax')(dot_product)  
    
    model = Model(inputs=[input_target, input_context], outputs=output)
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

def generate_skipgram_data(X_train_seq, vocab_size, window_size, negative_samples, batch_size):
    while True:
        X, y = [], []
        for seq in X_train_seq:
            pairs, labels = skipgrams(seq, vocabulary_size=vocab_size, window_size=window_size, negative_samples=negative_samples)
            if pairs:
                target_words, context_words = zip(*pairs)
                X.extend(zip(target_words, context_words))
                y.extend(labels)

            if len(X) >= batch_size:
                yield [np.array(X)[:, 0], np.array(X)[:, 1]], np.array(y)
                X, y = [], []




def prep_data(x_train):
    x_train = x_train.apply(remove_links)
    x_train = x_train .apply(remove_dates_and_times)
    x_train = x_train.apply(remove_numbers)
    x_train = x_train.apply(lambda x: ' '.join(remove_stop_words(x)))
    # x_train = x_train.apply(stem_arabic_words_snowballstemmer)
    x_train = x_train.apply(unify_hamzat)
    x_train = x_train.apply(unify_Alfat)
    x_train = x_train.apply(remove_tatweel)
    x_train = x_train.apply(remove_space)
    return x_train    

# Example usage:
embedding_dim = 50
window_size = 5
negative_samples = 5
batch_size = 128
vocab_size = 10000

tokenizer = Tokenizer(num_words=vocab_size)
# X_train_combined = prep_data(X_train_combined)
tokenizer.fit_on_texts(X_train_combined)

# total_words = len(tokenizer.word_index) + 1

# print(vocab_size)


X_train_seq = tokenizer.texts_to_sequences(X_train_combined)

model = build_skipgram_model(vocab_size, embedding_dim)
generator = generate_skipgram_data(X_train_seq, vocab_size, window_size, negative_samples, batch_size)

clear_session()
model.fit(generator, epochs=5, steps_per_epoch=len(X_train_seq) // batch_size)


Epoch 1/5
254/254 [==============================] - 152s 584ms/step - loss: 8.8956 - accuracy: 0.5990
Epoch 2/5
254/254 [==============================] - 134s 527ms/step - loss: 6.9999 - accuracy: 0.6410
Epoch 3/5
254/254 [==============================] - 130s 510ms/step - loss: 5.6593 - accuracy: 0.6563
Epoch 4/5
254/254 [==============================] - 192s 755ms/step - loss: 5.3157 - accuracy: 0.6444
Epoch 5/5
254/254 [==============================] - 141s 557ms/step - loss: 4.7400 - accuracy: 0.6726


In [265]:
# from keras.backend import clear_session
# clear_session()

## [6.4]

In [266]:
# # import matplotlib.pyplot as plt

# # embedding_layer = model.get_layer('embedding')
# # weights = embedding_layer.get_weights()[0]

# # word_index = tokenizer.word_index
# # reverse_word_index = {v: k for k, v in word_index.items()}

# # num_words_to_visualize = 100
# # words_to_visualize = [reverse_word_index[i] for i in range(1, num_words_to_visualize + 1)]

# # word_embeddings = weights[1:num_words_to_visualize + 1]

# # plt.figure(figsize=(10, 8))
# # for i, word in enumerate(words_to_visualize):
# #     plt.scatter(word_embeddings[i, 0], word_embeddings[i, 1])
# #     plt.text(word_embeddings[i, 0], word_embeddings[i, 1], word)

# # plt.title('Word Embeddings Visualization')
# # plt.xlabel('Dimension 1')
# # plt.ylabel('Dimension 2')
# # plt.show()


# import matplotlib.pyplot as plt
# from arabic_reshaper import arabic_reshaper
# from bidi.algorithm import get_display

# arabic_font_path = 'font/NotoNaskhArabic-VariableFont_wght.ttf'

# embedding_layer = model.get_layer('embedding')
# weights = embedding_layer.get_weights()[0]

# word_index = tokenizer.word_index
# reverse_word_index = {v: k for k, v in word_index.items()}

# num_words_to_visualize = 100
# words_to_visualize = [reverse_word_index[i] for i in range(1, num_words_to_visualize + 1)]

# word_embeddings = weights[1:num_words_to_visualize + 1]

# plt.figure(figsize=(10, 8))

# for i, word in enumerate(words_to_visualize):
#     reshaped_word = arabic_reshaper.reshape(word)
#     bidi_word = get_display(reshaped_word)

#     plt.scatter(word_embeddings[i, 0], word_embeddings[i, 1])
#     plt.text(word_embeddings[i, 0], word_embeddings[i, 1], bidi_word, fontproperties='Arial')

# arabic_font = FontProperties(fname='font/NotoNaskhArabic-VariableFont_wght.ttf')
# plt.title(get_display(arabic_reshaper.reshape('تصوير التضمين باللغة العربية')), fontproperties=arabic_font)
# plt.xlabel(get_display(arabic_reshaper.reshape('البُعد الأول')), fontproperties=arabic_font)
# plt.ylabel(get_display(arabic_reshaper.reshape('البُعد الثاني')), fontproperties=arabic_font)
# plt.show()
 


## [6.5]

In [267]:
# def analogy(word_a, word_b, word_c, model, tokenizer):
    
#     embeddings = model.get_layer('embedding').get_weights()[0]
#     word_index = tokenizer.word_index

#     if all(word in word_index for word in [word_a, word_b, word_c]):
#         a_idx, b_idx, c_idx = [word_index[word] for word in [word_a, word_b, word_c]]
#         vec_a, vec_b, vec_c = embeddings[a_idx], embeddings[b_idx], embeddings[c_idx]

#         # Calculate the analogy vector
#         analogy_vector = vec_b - vec_a + vec_c

#         # Find the closest word in the embedding space
#         closest_word_idx = np.argmin(np.linalg.norm(embeddings - analogy_vector, axis=1))

#         # Get the word from the index
#         closest_word = list(word_index.keys())[list(word_index.values()).index(closest_word_idx)]
#         return closest_word
#     else:
#         return None

# # Example usage:
# for _ in range(5):
#     word_a = "ضغط"
#     word_b = "علاج"
#     word_c = "التنفسي"

#     result = analogy(word_a, word_b, word_c, model, tokenizer)
#     print(f"{word_a} is to {word_b} as {word_c} is to {result}")


# Question [7]: Deep Neural Network

## [7.1]

In [268]:
# from keras.backend import clear_session
# clear_session()

In [269]:
X_train_combined ,X_test_combined,X_validation_combined = Prepare_data()
X_train ,y_train,X_test,y_test,X_validation,y_validation =Prepare()
X_train_combined=callfunction(X_train_combined)

### [7.1.1]

In [270]:
def convert_to_tf_sparse(X):
    coo = X.tocoo()
    indices = np.mat([coo.row, coo.col]).transpose()
    return tf.sparse.reorder(tf.sparse.SparseTensor(indices, coo.data.astype(np.float32), coo.shape))

In [271]:
# vectorizer = CountVectorizer()
# X_train_bow = vectorizer.fit_transform(X_train_combined)
# X_test_bow = vectorizer.transform(X_test_combined)
# X_validation_bow = vectorizer.transform(X_validation_combined)

# X_train_sparse = convert_to_tf_sparse(X_train_bow)
# X_test_sparse = convert_to_tf_sparse(X_test_bow)
# X_validation_sparse = convert_to_tf_sparse(X_validation_bow)

# label_encoder = LabelEncoder()
# y_train_encoded = label_encoder.fit_transform(y_train)
# y_test_encoded = label_encoder.transform(y_test)
# y_validation_encoded = label_encoder.transform(y_validation)

# y_train_categorical = to_categorical(y_train_encoded, num_classes=8)
# y_test_categorical = to_categorical(y_test_encoded, num_classes=8)
# y_validation_categorical = to_categorical(y_validation_encoded, num_classes=8)

# model = Sequential()
# model.add(Dense(64, activation='relu', input_dim=X_train_sparse.shape[1]))
# # model.add(Dropout(0.5))
# model.add(Dense(32, activation='relu'))
# # model.add(Dropout(0.5))
# model.add(Dense(8, activation='softmax')) 

# model.compile(optimizer='adam',
#               loss='categorical_crossentropy',
#               metrics=['accuracy'])

# model.fit(X_train_sparse, y_train_categorical, epochs=10, batch_size=32, validation_data=(X_validation_sparse, y_validation_categorical))

# loss, accuracy = model.evaluate(X_test_sparse, y_test_categorical)
# print(f'Loss: {loss}, Accuracy: {accuracy}')

In [272]:
# comparison_table['question_step_number'].append("7.1.1")
# comparison_table['model_name'].append('Deep Neural Network')
# comparison_table['features'].append('BOW')
# comparison_table['model_parameters'].append('default')
# comparison_table['preprocessing_methods'].append("callfunction")
# comparison_table['accuracy'].append(accuracy)
# comparison_table['balance_accuracy'].append(loss)

### [7.1.2]

In [273]:
# vectorizer  = TfidfVectorizer(min_df=2,max_df=0.5,ngram_range=(1,2))

# train_data_X_train_combined = np.array(X_train_combined)
# train_data_X_test_combined = np.array(X_test_combined)
# train_data_X_validation_combined = np.array(X_validation_combined)
# tfidf_matrix_X_train_combined = vectorizer .fit_transform(train_data_X_train_combined)
# tfidf_matrix_X_test_combined = vectorizer .transform(train_data_X_test_combined)
# tfidf_matrix_X_validation_combined = vectorizer .transform(train_data_X_validation_combined)

# tfidf_matrix_X_train_combined=convert_to_tf_sparse(tfidf_matrix_X_train_combined)
# tfidf_matrix_X_test_combined=convert_to_tf_sparse(tfidf_matrix_X_test_combined)
# tfidf_matrix_X_validation_combined=convert_to_tf_sparse(tfidf_matrix_X_validation_combined)

# label_encoder = LabelEncoder()
# y_train_encoded = label_encoder.fit_transform(y_train)
# y_test_encoded = label_encoder.transform(y_test)
# y_validation_encoded = label_encoder.transform(y_validation)

# y_train_categorical = to_categorical(y_train_encoded, num_classes=8)
# y_test_categorical = to_categorical(y_test_encoded, num_classes=8)
# y_validation_categorical = to_categorical(y_validation_encoded, num_classes=8)


# model = Sequential()
# model.add(Dense(64, activation='relu', input_dim=tfidf_matrix_X_train_combined.shape[1]))
# model.add(Dropout(0.5))
# model.add(Dense(32, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(y_train_categorical.shape[1], activation='softmax')) 

# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# model.fit(tfidf_matrix_X_train_combined, y_train_categorical, epochs=10, batch_size=32,validation_data=(tfidf_matrix_X_validation_combined, y_validation_categorical))

# loss, accuracy = model.evaluate(tfidf_matrix_X_test_combined, y_test_categorical)
# print(f'Loss: {loss}, Accuracy: {accuracy}')

In [274]:
# comparison_table['question_step_number'].append("7.1.2")
# comparison_table['model_name'].append('Deep Neural Network')
# comparison_table['features'].append('TF-IDF')
# comparison_table['model_parameters'].append('default')
# comparison_table['preprocessing_methods'].append("callfunction")
# comparison_table['accuracy'].append(accuracy)
# comparison_table['balance_accuracy'].append(loss)

## [7.2]

### [7.2.1]

In [298]:
embedding_layer_weights = model.layers[2].get_weights()[0]

In [303]:
# from keras.models import Sequential
# from keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D, Dense
# from keras.initializers import Constant

# # فرضاً لدينا مصفوفة الأشعة من Word2Vec
# word_vectors = embedding_layer_weights

# # إنشاء النموذج
# cnn_model_1 = Sequential()
# cnn_model_1.add(Embedding(input_dim=word_vectors.shape[0],
#                     output_dim=word_vectors.shape[1],
#                     embeddings_initializer=Constant(word_vectors),
#                     trainable=False))  
# cnn_model_1.add(Conv1D(128, 5, activation='relu'))
# cnn_model_1.add(MaxPooling1D(5))
# cnn_model_1.add(Conv1D(128, 5, activation='relu'))
# cnn_model_1.add(GlobalMaxPooling1D())
# cnn_model_1.add(Dense(8, activation='softmax')) 

# # تجميع النموذج
# cnn_model_1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


# # model.fit(x_train, y_train, batch_size=32, epochs=10)
# cnn_model_1.fit(X_train, y_train, epochs=5, batch_size=32, validation_split=0.2)

from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense
from keras.initializers import Constant
from keras.preprocessing.text import Tokenizer
from keras.backend import clear_session
import numpy as np
import pandas as pd

# Assuming your training data (X_train_combined) and labels (y_train)
# Assuming you have trained your Skip-gram model as per your previous code

# Assuming your sequence length is 5 and embedding dimension is 50 (increased sequence length)
sequence_length = 5
embedding_dim = 50

# Assuming you have the vocabulary size and word vectors from the Skip-gram model
vocab_size = 10000
word_vectors = model.layers[2].get_weights()[0]  # Assuming model is your trained Skip-gram model

# Create and compile the CNN model
cnn_model_1 = Sequential()
cnn_model_1.add(Embedding(input_dim=vocab_size,
                          output_dim=embedding_dim,
                          embeddings_initializer=Constant(word_vectors),
                          input_length=sequence_length,
                          trainable=False))  # Fix the pre-trained word vectors
cnn_model_1.add(Conv1D(128, 5, activation='relu'))
cnn_model_1.add(GlobalMaxPooling1D())
cnn_model_1.add(Dense(8, activation='softmax')) 

# Compile the model
cnn_model_1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Assuming you have your training data (X_train_combined) and labels (y_train)
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(X_train_combined)
X_train_seq = tokenizer.texts_to_sequences(X_train_combined)

# Assuming your y_train is one-hot encoded
y_onehot1 = pd.get_dummies(train_data['lable'])
y_train = y_onehot1.values  # Convert to numpy array

# Assuming your X_train_seq is a list of sequences
X_train_seq_padded = pad_sequences(X_train_seq, maxlen=sequence_length, padding='post', truncating='post')

# Print shapes for debugging
print("X_train_seq_padded shape:", X_train_seq_padded.shape)

# Train the CNN model
cnn_model_1.fit(X_train_seq_padded, y_train, epochs=5, batch_size=32, validation_split=0.2)


X_train_seq_padded shape: (32598, 5)
Epoch 1/5
815/815 [==============================] - 9s 8ms/step - loss: 1.9806 - accuracy: 0.1983 - val_loss: 1.9867 - val_accuracy: 0.2055
Epoch 2/5
815/815 [==============================] - 6s 8ms/step - loss: 1.9719 - accuracy: 0.2052 - val_loss: 1.9850 - val_accuracy: 0.2118
Epoch 3/5
815/815 [==============================] - 6s 7ms/step - loss: 1.9700 - accuracy: 0.2043 - val_loss: 1.9823 - val_accuracy: 0.2100
Epoch 4/5
815/815 [==============================] - 7s 9ms/step - loss: 1.9690 - accuracy: 0.2062 - val_loss: 1.9841 - val_accuracy: 0.1969
Epoch 5/5
815/815 [==============================] - 6s 8ms/step - loss: 1.9681 - accuracy: 0.2037 - val_loss: 1.9806 - val_accuracy: 0.2132


### [7.2.2]

In [ ]:
# فرضاً لدينا مصفوفة الأشعة من Word2Vec
word_vectors = # مصفوفة الأشعة من Word2Vec

# إنشاء النموذج
cnn_model_2 = Sequential()
cnn_model_2.add(Embedding(input_dim=word_vectors.shape[0],
                    output_dim=word_vectors.shape[1],
                    embeddings_initializer=Constant(word_vectors),
                    trainable=True))  # السماح بتعديل طبقة التضمين
cnn_model_2.add(Conv1D(128, 5, activation='relu'))
cnn_model_2.add(MaxPooling1D(5))
cnn_model_2.add(Conv1D(128, 5, activation='relu'))
cnn_model_2.add(GlobalMaxPooling1D())
cnn_model_2.add(Dense(8, activation='softmax'))  # 8 عقد للتصنيف الثماني

# تجميع النموذج
cnn_model_2.compile(optimizer='adam',
               loss='categorical_crossentropy',
                 metrics=['accuracy'])

# تدريب النموذج (استبدل x_train و y_train ببياناتك)
# model.fit(x_train, y_train, batch_size=32, epochs=10)
cnn_model_2.fit(X_train, y_train, epochs=5, batch_size=32, validation_split=0.2)


### [7.2.3]

In [ ]:
# فرضاً لدينا حجم الفوكابولاري وأبعاد الأشعة
vocab_size = # حجم الفوكابولاري
embedding_dim = # أبعاد الأشعة

# إنشاء النموذج
cnn_model_3 = Sequential()
cnn_model_3.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim))
cnn_model_3.add(Conv1D(128, 5, activation='relu'))
cnn_model_3.add(MaxPooling1D(5))
cnn_model_3.add(Conv1D(128, 5, activation='relu'))
cnn_model_3.add(GlobalMaxPooling1D())
cnn_model_3.add(Dense(8, activation='softmax'))  # 8 عقد للتصنيف الثماني

# تجميع النموذج
cnn_model_3.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# تدريب النموذج (استبدل x_train و y_train ببياناتك)
# model.fit(x_train, y_train, batch_size=32, epochs=10)
cnn_model_3.fit(X_train, y_train, epochs=5, batch_size=32, validation_split=0.2)


### [7.2.4]

In [ ]:
# تحميل مصفوفة التضمين (مثال فقط، استبدلها بمصفوفتك)
embeddings_index = # تحميل مصفوفة التضمين

# إنشاء النموذج
cnn_model_4 = Sequential()
cnn_model_4.add(Embedding(input_dim=len(embeddings_index), 
                    output_dim=embedding_dim, 
                    embeddings_initializer=Constant(embeddings_index),
                    trainable=False))  # جمد طبقة التضمين
cnn_model_4.add(Conv1D(128, 5, activation='relu'))
cnn_model_4.add(MaxPooling1D(5))
cnn_model_4.add(Conv1D(128, 5, activation='relu'))
cnn_model_4.add(GlobalMaxPooling1D())
cnn_model_4.add(Dense(8, activation='softmax'))  # على سبيل المثال، لـ 8 فئات

# تجميع النموذج
cnn_model_4.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# تدريب النموذج (استبدل x_train و y_train ببياناتك)
# model.fit(x_train, y_train, batch_size=32, epochs=10)
cnn_model_4.fit(X_train, y_train, epochs=5, batch_size=32, validation_split=0.2)


## [7.3]

## Extra [7.4]

# Final Results

In [ ]:
df = pd.DataFrame(comparison_table)
df

In [ ]:
df.to_csv("[your_name].csv", index=False)